## 0. Introduction

The aim of this lab is to get familiar with **Mixture of Gaussians** model.


1.   This lab is the course-work activity **Assignment 2: Unsupervised Learning(20%)**
2. The Assignment is due on **Friday 29th November (6pm)**
2.   A report answering the <font color = 'red'>**questions in</font><font color = "red"> red**</font> should be submitted on QMplus along with the completed Notebook.
3. The report should be a separate file in **pdf format** (so **NOT** *doc, docx, notebook* etc.), well identified with your name, student number, assignment number (for instance, Assignment 2), module code.
4. Make sure that **any figures or code** you comment on, are **included in the report**.
5. You should submit a zip file that contains 1 x Notebook and 1 x PDF
6. No other means of submission other than the appropriate QM+ link is acceptable at any time (so NO email attachments, etc.)
7. **PLAGIARISM** <ins>is an irreversible non-negotiable failure in the course</ins> (if in doubt of what constitutes plagiarism, ask!).

In [ ]:
pip install numpy


In [116]:
import os
import numpy as np
import scipy
from sklearn.preprocessing import normalize
import matplotlib.pyplot as plt
from IPython import display
%matplotlib inline

For this lab, we will use the Peterson and Barney’s dataset of vowel formant frequencies. (For more info, look at Classification of Peterson & Barney’s vowels using Weka. - a copy of this article is on QMplus)

More specifically, Peterson and Barney measured the fundamental frequency $F0$ and the first three formant frequencies ($F1-F3$) of sustained English Vowels, using samples from various speakers.

Upload the .npy file and load it using the code block below.

In [ ]:
import numpy as np

# Specify the filename
data_npy_file = 'PB_data.npy'  # Ensure this file is in your working directory

# Load the data from .npy file
data = np.load(data_npy_file, allow_pickle=True)

print(data)  # Display the data or perform your operations

In [118]:
# from google.colab import files
# files.upload()
# data_npy_file = 'PB_data.npy'

# # Loading data from .npy file
# data = np.load(data_npy_file, allow_pickle=True)

In [ ]:
data

The dataset contains 4 vectors ($F0-F3$), containing the fundamental frequencies ($F0$, $F1$, $F2$ and $F3$) for each phoneme and another vector “phoneme\_id” containing a number representing the id of the phoneme.

In the exercises that follow, we will use only the dataset associated with formants $F1$ and $F2$.

**Note**: Phonemes and Frequencies are two different things; phoneme_id is the ground truth class of each row while frequencies represent features of each data point. Please review the papers provided with the lab materials for more details.

In [120]:
data = np.ndarray.tolist(data)
# Array that contains the phoneme ID (1-10) of each sample
phoneme_id = data['phoneme_id']

# frequencies f1 and f2
f1 = data['f1']
f2 = data['f2']

In [ ]:
for it in data['phoneme_id']:
    print(it)

In [ ]:
f1.shape

In [ ]:
f2.shape

### <font color="red"> Q1. Produce a plot of $F1$ against $F2$. (You should be able to spot some clusters already in this scatter plot.). Comment on the figure and the visible clusters [2 marks]</font>

In [ ]:
# Create scatter plot of F1 vs F2
plt.figure(figsize=(10,8))

# Create subplot with 2 axes
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(10,8))

# First plot without colors
ax1.scatter(f1, f2, alpha=0.5, color='blue')
ax1.set_xlabel('F1 Frequency')
ax1.set_ylabel('F2 Frequency')
ax1.set_title('F1 vs F2 Frequencies')
ax1.grid(True)

# Second plot with phoneme colors
scatter = ax2.scatter(f1, f2, c=phoneme_id, cmap='tab10', alpha=0.5)
ax2.set_xlabel('F1 Frequency') 
ax2.set_ylabel('F2 Frequency')
ax2.set_title('F1 vs F2 Frequencies by Phoneme')
plt.colorbar(scatter, ax=ax2, label='Phoneme ID')
ax2.grid(True)

plt.tight_layout()
plt.show()

# The scatter plots show several visible clusters of phonemes based on their F1 and F2 frequencies
# We can observe:
# - Some clear separation between different phoneme groups, especially visible in the colored plot
# - Overlap between certain phoneme clusters indicating similar frequency characteristics
# - Most phonemes appear to form elliptical clusters suggesting Gaussian-like distributions
# - The F1 frequencies generally range from ~200-1000 Hz while F2 spans ~1000-3000 Hz
# - Having both plots side by side helps visualize the natural clustering vs phoneme groupings

# 1. MoG Using the EM algorithm

Recall the following definition of a Mixture of Gaussians. Assuming our observed random vector is $\mathbf{x}\in\mathbb{R}^D$, a MoG models $p(\mathbf{x})$ as a sum of $K$-many weighted Gaussians. More specifically:


\begin{equation}
 p(\mathbf{x}) = \sum_{k=1}^K \frac{p(c_{k})}{(2\pi)^{D/2} \mathrm{det}\left(\boldsymbol\Sigma_k\right)^{1/2}}
 % \exp(-\frac{1}{2}(x-\mu)^T \sum_{k}^{-1}(x-\mu))
 \exp\left(-\frac{1}{2}(\mathbf{x}-\boldsymbol\mu_k)^\top {\boldsymbol\Sigma_k}^{-1}(\mathbf{x}-\boldsymbol\mu_k)\right),
\end{equation}
where:
* $\boldsymbol\mu_k\in\mathbb{R}^{D}$
* $\boldsymbol\Sigma_{k}\in\mathbb{R}^{D\times D}$
* $p(c_{k})=\pi_k\in\mathbb{R}$

denote the $k$-th gaussian component's **mean vector**, **covariance matrix**, and **mixture coefficients** respectively. The $K$-many gaussian components' model parameters are referred to collectively as $\theta = \{\boldsymbol\mu_k,\boldsymbol\Sigma_{k},\pi_k\}_{k=1}^K$.

## EM Algorithm
For the E step we softly assign each datum to the closest centroid (using the current iteration's fixed model parameters) as in the K means example.

For the M step we update the model parameters $\theta$ to maximize the weighted log-likelihood.
At a high level, for each centroid k we:
* Update the mean vectors
* Update the covariance matrices (We will fit Gaussians with diagonal covariance matrices)
* Set the mixture coefficients as the mean probability of a sample being generated by the k-th gaussian component

Read the code below and understand what it is calculating

In [125]:
def get_predictions(mu, s, p, X):
	"""
		:param mu			: means of GMM components
		:param s			: covariances of GMM components
		:param p 			: weights of GMM components
		:param X			: 2D array of our dataset
	"""

	# get number of GMM components
	k = s.shape[0]
	# get number of data samples
	N = X.shape[0]
	# get dimensionality of our dataset
	D = X.shape[1]

	Z = np.zeros((N, k))
	for i in range(k):
		mu_i = mu[i, :]
		mu_i = np.expand_dims(mu_i, axis=1)
		mu_i_repeated = np.repeat(mu_i, N, axis=1)
		X_minus_mu = X - mu_i_repeated.transpose()
		inverse_s = scipy.linalg.pinv(s[i])
		inverse_s = np.squeeze(inverse_s)
		s_i_det = scipy.linalg.det(s[i])
		x_s_x = np.matmul(X_minus_mu, inverse_s)*X_minus_mu
		Z[:, i] = p[i]*(1/np.power(((2*np.pi)**D) * np.abs(s_i_det), 0.5)) * np.exp(-0.5*np.sum(x_s_x, axis=1))
	return Z


### Helper functions

In [126]:
def plot_gaussians(ax, s, mu, X, title_string):
    """
        :param X            : 2D array of our dataset
        :param y            : 1D array of the groundtruth labels of the dataset
        :param ax          : existing subplot, to draw on it
    """
    ax.clear()
    if X.shape[1] == 2:
        # set label of horizontal axis
        ax.set_xlabel('f1')
        # set label of vertical axis
        ax.set_ylabel('f2')
        # scatter the points, with red color
        ax.scatter(X[:,0], X[:,1], c='black', marker='.', label=title_string)

    k = s.shape[0]
    color_list = ['r', 'g', 'b', 'c', 'm', 'y']
    N_sides = 11

    # Iterate over all gaussians
    for k_cnt in range(k):

        # pick the covariance matrix and mean values of each gaussian
        s_k = s[k_cnt]
        mu_k = mu[k_cnt]

        if s_k.shape[0]==2:
            # dataset with 2 features
            side_range = np.arange(0, N_sides, 1)
            theta = side_range/(N_sides-1)*2*np.pi
            matrix = np.array([np.cos(theta), np.sin(theta)])
            mu_k = np.expand_dims(mu_k, axis=1)
            mu_repeated = np.repeat(mu_k, N_sides, axis=1)
            epoints = np.matmul(scipy.linalg.sqrtm(s_k), matrix) + mu_repeated
            plt.plot(epoints[0,:], epoints[1,:], color=color_list[k_cnt], linewidth=2)
            plt.scatter(mu_k[0], mu_k[1], color=color_list[k_cnt], marker='o', linewidth=2)
        elif s_k.shape[0]==3:
            # dataset with 3 features
            side_range = np.arange(0, N_sides, 1)
            theta = side_range/(N_sides-1)*np.pi
            phi = side_range/(N_sides-1)*2*np.pi
            sin_theta = np.expand_dims(np.sin(theta), axis=1)
            cos_theta = np.expand_dims(np.cos(theta), axis=1)
            sin_phi = np.expand_dims(np.sin(phi), axis=1)
            cos_phi = np.expand_dims(np.cos(phi), axis=1)
            sx = np.matmul(sin_theta, cos_phi.transpose())
            sy = np.matmul(sin_theta, sin_phi.transpose())
            sz = np.matmul(cos_theta, np.ones((1, N_sides)))
            svect = np.array([sx.reshape((N_sides*N_sides)), sy.reshape((N_sides*N_sides)), sz.reshape((N_sides*N_sides))])

            mu_k = np.expand_dims(mu_k, axis=1)
            mu_repeated = np.repeat(mu_k, N_sides*N_sides, axis=1)
            epoints = np.matmul(scipy.linalg.sqrtm(s_k), svect) + mu_repeated

            ex = epoints[0]
            ey = epoints[1]
            ez = epoints[2]

            ax.plot3D(epoints[0,:], epoints[1,:], epoints[2,:], color=color_list[k_cnt])
            ax.plot3D(ex, ey, ez, color=color_list[k_cnt])
        else:
            print('Each dataset sample should have either 2 or 3 features. Not plotting this one.')
    ax.legend()
    display.clear_output(wait=True)
    display.display(plt.gcf())

def plot_data_3D(X, title_string, ax):
    """
        :param X            : 2D array of our dataset
        :param y            : 1D array of the groundtruth labels of the dataset
        :param ax          : existing subplot, to draw on it
    """
    # clear subplot from previous (if any) drawn stuff
    ax.clear()
    # set label of x axis
    ax.set_xlabel('f1')
    # set label of y axis
    ax.set_ylabel('f2')
    # set label of z axis
    ax.set_zlabel('f1 + f2')
    # scatter the points, with red color
    ax.scatter3D(X[:,0], X[:,1], X[:,2], c='black', marker='.', label=title_string)
    # add legend to the subplot
    ax.legend()


def save_model(mu, s, p):
  GMM_parameters = {}
  GMM_parameters['mu'] = mu
  GMM_parameters['s'] = s
  GMM_parameters['p'] = p
  if not os.path.isdir('data/'):
    os.mkdir('data/')
  npy_filename = 'data/GMM_params_phoneme_{:02}_k_{:02}.npy'.format(p_id, k)
  np.save(npy_filename, GMM_parameters)

## Train MoG

Read the following blocks of code and understand what it does.

In [127]:
import numpy as np

k = 3
# id of the phoneme that will be used (e.g. 1, or 2)
p_id = 1
X_full = np.zeros((len(f1), 2))
X_full[:,0] = f1.copy()
X_full[:,1] = f2.copy()
X_full = X_full.astype(np.float32)

X = X_full[phoneme_id==p_id,:]

In [ ]:
# get number of samples
N = X.shape[0]
# get dimensionality of our dataset
D = X.shape[1]
# common practice : GMM weights initially set as 1/k
p = np.ones((k))/k
# GMM means are picked randomly from data samples
random_indices = np.floor(N*np.random.rand((k)))
random_indices = random_indices.astype(int)
mu = X[random_indices,:] # shape kxD
# covariance matrices
s = np.zeros((k,D,D)) # shape kxDxD
# number of iterations for the EM algorithm
n_iter = 100

# initialize covariances
for i in range(k):
    cov_matrix = np.cov(X.transpose())
    # initially set to fraction of data covariance
    s[i,:,:] = cov_matrix/k

# Initialize array Z that will get the predictions of each Gaussian on each sample
Z = np.zeros((N,k)) # shape Nxk

###############################
# run Expectation Maximization algorithm for n_iter iterations
n_iter = 100
for t in range(n_iter):
    print('Iteration {:03}/{:03}'.format(t+1, n_iter))

    # Do the E-step
    Z = get_predictions(mu, s, p, X)
    Z = normalize(Z, axis=1, norm='l1')

    # Do the M-step:
    for i in range(k):
        mu[i,:] = np.matmul(X.transpose(),Z[:,i]) / np.sum(Z[:,i])

        ###################################################
        # We will fit Gaussians with diagonal covariance matrices:
        mu_i = mu[i,:]
        mu_i = np.expand_dims(mu_i, axis=1)
        mu_i_repeated = np.repeat(mu_i, N, axis=1)
        coef_1 = (X.transpose() - mu_i_repeated)**2
        coef_2 = Z[:,i]/np.sum(Z[:,i])
        coef_2 = np.expand_dims(coef_2, axis=1)
        res_1 = np.squeeze( np.matmul(coef_1, coef_2) )
        s[i,:,:] = np.diag(res_1)

        p[i] = np.mean(Z[:,i])

    fig, ax1 = plt.subplots()
    plot_gaussians(ax1, 2*s, mu, X, 'Phoneme {}'.format(p_id))
print('\nFinished.\n')
print(mu, s)
save_model(mu, s, p)

### <font color="red"> Q2. Run the code multiple times for $K=3$, what do you observe? Use figures and the printed MoG parameters to support your arguments [2 mark] </font>
###  Q3. Repeat the training for phoneme_id = 2  and save the parameters [1 mark]
### Q4. Repeat training for $K=6$ for phoneme_id =1 and phoneme_id=2 and save the model parameters [1 mark]

Save your MoG model: this should comprise the variables mu, s and p. Include the all the saved model parameters in the submission .zip folder.

## Q2

In [145]:
import numpy as np

k = 3
# id of the phoneme that will be used (e.g. 1, or 2)
p_id = 1
X_full = np.zeros((len(f1), 2))
X_full[:,0] = f1.copy()
X_full[:,1] = f2.copy()
X_full = X_full.astype(np.float32)

X = X_full[phoneme_id==p_id,:]

## Iter 1

In [ ]:
# get number of samples
N = X.shape[0]
# get dimensionality of our dataset
D = X.shape[1]
# common practice : GMM weights initially set as 1/k
p = np.ones((k))/k
# GMM means are picked randomly from data samples
random_indices = np.floor(N*np.random.rand((k)))
random_indices = random_indices.astype(int)
mu = X[random_indices,:] # shape kxD
# covariance matrices
s = np.zeros((k,D,D)) # shape kxDxD
# number of iterations for the EM algorithm
n_iter = 100

# initialize covariances
for i in range(k):
    cov_matrix = np.cov(X.transpose())
    # initially set to fraction of data covariance
    s[i,:,:] = cov_matrix/k

# Initialize array Z that will get the predictions of each Gaussian on each sample
Z = np.zeros((N,k)) # shape Nxk

###############################
# run Expectation Maximization algorithm for n_iter iterations
n_iter = 100
for t in range(n_iter):
    print('Iteration {:03}/{:03}'.format(t+1, n_iter))

    # Do the E-step
    Z = get_predictions(mu, s, p, X)
    Z = normalize(Z, axis=1, norm='l1')

    # Do the M-step:
    for i in range(k):
        mu[i,:] = np.matmul(X.transpose(),Z[:,i]) / np.sum(Z[:,i])

        ###################################################
        # We will fit Gaussians with diagonal covariance matrices:
        mu_i = mu[i,:]
        mu_i = np.expand_dims(mu_i, axis=1)
        mu_i_repeated = np.repeat(mu_i, N, axis=1)
        coef_1 = (X.transpose() - mu_i_repeated)**2
        coef_2 = Z[:,i]/np.sum(Z[:,i])
        coef_2 = np.expand_dims(coef_2, axis=1)
        res_1 = np.squeeze( np.matmul(coef_1, coef_2) )
        s[i,:,:] = np.diag(res_1)

        p[i] = np.mean(Z[:,i])

    fig, ax1 = plt.subplots()
    plot_gaussians(ax1, 2*s, mu, X, 'Phoneme {}'.format(p_id))
print('\nFinished.\n')
# print(mu, s)
print(f'mean \n {mu}')
print(f'covariance \n {s}')
print(f'weight \n {p}')
save_model(mu, s, p)

## Iter 2

In [ ]:
# get number of samples
N = X.shape[0]
# get dimensionality of our dataset
D = X.shape[1]
# common practice : GMM weights initially set as 1/k
p = np.ones((k))/k
# GMM means are picked randomly from data samples
random_indices = np.floor(N*np.random.rand((k)))
random_indices = random_indices.astype(int)
mu = X[random_indices,:] # shape kxD
# covariance matrices
s = np.zeros((k,D,D)) # shape kxDxD
# number of iterations for the EM algorithm
n_iter = 100

# initialize covariances
for i in range(k):
    cov_matrix = np.cov(X.transpose())
    # initially set to fraction of data covariance
    s[i,:,:] = cov_matrix/k

# Initialize array Z that will get the predictions of each Gaussian on each sample
Z = np.zeros((N,k)) # shape Nxk

###############################
# run Expectation Maximization algorithm for n_iter iterations
n_iter = 100
for t in range(n_iter):
    print('Iteration {:03}/{:03}'.format(t+1, n_iter))

    # Do the E-step
    Z = get_predictions(mu, s, p, X)
    Z = normalize(Z, axis=1, norm='l1')

    # Do the M-step:
    for i in range(k):
        mu[i,:] = np.matmul(X.transpose(),Z[:,i]) / np.sum(Z[:,i])

        ###################################################
        # We will fit Gaussians with diagonal covariance matrices:
        mu_i = mu[i,:]
        mu_i = np.expand_dims(mu_i, axis=1)
        mu_i_repeated = np.repeat(mu_i, N, axis=1)
        coef_1 = (X.transpose() - mu_i_repeated)**2
        coef_2 = Z[:,i]/np.sum(Z[:,i])
        coef_2 = np.expand_dims(coef_2, axis=1)
        res_1 = np.squeeze( np.matmul(coef_1, coef_2) )
        s[i,:,:] = np.diag(res_1)

        p[i] = np.mean(Z[:,i])

    fig, ax1 = plt.subplots()
    plot_gaussians(ax1, 2*s, mu, X, 'Phoneme {}'.format(p_id))
print('\nFinished.\n')
# print(mu, s)
print(f'mean \n {mu}')
print(f'covariance \n {s}')
print(f'weight \n {p}')
save_model(mu, s, p)

## Q3

In [132]:
k = 3
# id of the phoneme that will be used (e.g. 1, or 2)
p_id = 2
X_full = np.zeros((len(f1), 2))
X_full[:,0] = f1.copy()
X_full[:,1] = f2.copy()
X_full = X_full.astype(np.float32)

X = X_full[phoneme_id==p_id,:]

In [ ]:
# get number of samples
N = X.shape[0]
# get dimensionality of our dataset
D = X.shape[1]
# common practice : GMM weights initially set as 1/k
p = np.ones((k))/k
# GMM means are picked randomly from data samples
random_indices = np.floor(N*np.random.rand((k)))
random_indices = random_indices.astype(int)
mu = X[random_indices,:] # shape kxD
# covariance matrices
s = np.zeros((k,D,D)) # shape kxDxD
# number of iterations for the EM algorithm
n_iter = 100

# initialize covariances
for i in range(k):
    cov_matrix = np.cov(X.transpose())
    # initially set to fraction of data covariance
    s[i,:,:] = cov_matrix/k

# Initialize array Z that will get the predictions of each Gaussian on each sample
Z = np.zeros((N,k)) # shape Nxk

###############################
# run Expectation Maximization algorithm for n_iter iterations
n_iter = 100
for t in range(n_iter):
    print('Iteration {:03}/{:03}'.format(t+1, n_iter))

    # Do the E-step
    Z = get_predictions(mu, s, p, X)
    Z = normalize(Z, axis=1, norm='l1')

    # Do the M-step:
    for i in range(k):
        mu[i,:] = np.matmul(X.transpose(),Z[:,i]) / np.sum(Z[:,i])

        ###################################################
        # We will fit Gaussians with diagonal covariance matrices:
        mu_i = mu[i,:]
        mu_i = np.expand_dims(mu_i, axis=1)
        mu_i_repeated = np.repeat(mu_i, N, axis=1)
        coef_1 = (X.transpose() - mu_i_repeated)**2
        coef_2 = Z[:,i]/np.sum(Z[:,i])
        coef_2 = np.expand_dims(coef_2, axis=1)
        res_1 = np.squeeze( np.matmul(coef_1, coef_2) )
        s[i,:,:] = np.diag(res_1)

        p[i] = np.mean(Z[:,i])

    fig, ax1 = plt.subplots()
    plot_gaussians(ax1, 2*s, mu, X, 'Phoneme {}'.format(p_id))
print('\nFinished.\n')
print(mu, s)
save_model(mu, s, p)

## Q4

#### K=6 and Phoneme_ID = 1

In [ ]:
k = 6
# id of the phoneme that will be used (e.g. 1, or 2)
p_id = 1
X_full = np.zeros((len(f1), 2))
X_full[:,0] = f1.copy()
X_full[:,1] = f2.copy()
X_full = X_full.astype(np.float32)

X = X_full[phoneme_id==p_id,:]

# get number of samples
N = X.shape[0]
# get dimensionality of our dataset
D = X.shape[1]
# common practice : GMM weights initially set as 1/k
p = np.ones((k))/k
# GMM means are picked randomly from data samples
random_indices = np.floor(N*np.random.rand((k)))
random_indices = random_indices.astype(int)
mu = X[random_indices,:] # shape kxD
# covariance matrices
s = np.zeros((k,D,D)) # shape kxDxD
# number of iterations for the EM algorithm
n_iter = 100

# initialize covariances
for i in range(k):
    cov_matrix = np.cov(X.transpose())
    # initially set to fraction of data covariance
    s[i,:,:] = cov_matrix/k

# Initialize array Z that will get the predictions of each Gaussian on each sample
Z = np.zeros((N,k)) # shape Nxk

###############################
# run Expectation Maximization algorithm for n_iter iterations
n_iter = 100
for t in range(n_iter):
    print('Iteration {:03}/{:03}'.format(t+1, n_iter))

    # Do the E-step
    Z = get_predictions(mu, s, p, X)
    Z = normalize(Z, axis=1, norm='l1')

    # Do the M-step:
    for i in range(k):
        mu[i,:] = np.matmul(X.transpose(),Z[:,i]) / np.sum(Z[:,i])

        ###################################################
        # We will fit Gaussians with diagonal covariance matrices:
        mu_i = mu[i,:]
        mu_i = np.expand_dims(mu_i, axis=1)
        mu_i_repeated = np.repeat(mu_i, N, axis=1)
        coef_1 = (X.transpose() - mu_i_repeated)**2
        coef_2 = Z[:,i]/np.sum(Z[:,i])
        coef_2 = np.expand_dims(coef_2, axis=1)
        res_1 = np.squeeze( np.matmul(coef_1, coef_2) )
        s[i,:,:] = np.diag(res_1)

        p[i] = np.mean(Z[:,i])

    fig, ax1 = plt.subplots()
    plot_gaussians(ax1, 2*s, mu, X, 'Phoneme {}'.format(p_id))
print('\nFinished.\n')
print(mu, s)
save_model(mu, s, p)

#### K=6 and phoneme_ID = 2

In [ ]:
k = 6
# id of the phoneme that will be used (e.g. 1, or 2)
p_id = 2
X_full = np.zeros((len(f1), 2))
X_full[:,0] = f1.copy()
X_full[:,1] = f2.copy()
X_full = X_full.astype(np.float32)

X = X_full[phoneme_id==p_id,:]

# get number of samples
N = X.shape[0]
# get dimensionality of our dataset
D = X.shape[1]
# common practice : GMM weights initially set as 1/k
p = np.ones((k))/k
# GMM means are picked randomly from data samples
random_indices = np.floor(N*np.random.rand((k)))
random_indices = random_indices.astype(int)
mu = X[random_indices,:] # shape kxD
# covariance matrices
s = np.zeros((k,D,D)) # shape kxDxD
# number of iterations for the EM algorithm
n_iter = 100

# initialize covariances
for i in range(k):
    cov_matrix = np.cov(X.transpose())
    # initially set to fraction of data covariance
    s[i,:,:] = cov_matrix/k

# Initialize array Z that will get the predictions of each Gaussian on each sample
Z = np.zeros((N,k)) # shape Nxk

###############################
# run Expectation Maximization algorithm for n_iter iterations
n_iter = 100
for t in range(n_iter):
    print('Iteration {:03}/{:03}'.format(t+1, n_iter))

    # Do the E-step
    Z = get_predictions(mu, s, p, X)
    Z = normalize(Z, axis=1, norm='l1')

    # Do the M-step:
    for i in range(k):
        mu[i,:] = np.matmul(X.transpose(),Z[:,i]) / np.sum(Z[:,i])

        ###################################################
        # We will fit Gaussians with diagonal covariance matrices:
        mu_i = mu[i,:]
        mu_i = np.expand_dims(mu_i, axis=1)
        mu_i_repeated = np.repeat(mu_i, N, axis=1)
        coef_1 = (X.transpose() - mu_i_repeated)**2
        coef_2 = Z[:,i]/np.sum(Z[:,i])
        coef_2 = np.expand_dims(coef_2, axis=1)
        res_1 = np.squeeze( np.matmul(coef_1, coef_2) )
        s[i,:,:] = np.diag(res_1)

        p[i] = np.mean(Z[:,i])

    fig, ax1 = plt.subplots()
    plot_gaussians(ax1, 2*s, mu, X, 'Phoneme {}'.format(p_id))
print('\nFinished.\n')
print(mu, s)
save_model(mu, s, p)

### <font color="red"> Q5. Use the 2 MoGs ($K=3$) learnt in tasks 2 & 3 to build a classifier to discriminate between phonemes 1 and 2, and explain the process in the report [4 marks] </font>

Classify using the Maximum Likelihood (ML) criterion and calculate the miss-classification error. Remember that a classification under the ML compares $p(\mathbf{x};\boldsymbol\theta_{1})$, where $\boldsymbol\theta_{1}$ are the parameters of the MoG learnt for the first phoneme, with $p(\mathbf{x};\boldsymbol\theta_{2})$, where $\boldsymbol\theta_{2}$  are the parameters of the MoG learnt for the second phoneme.

In [ ]:
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

# Scatter Plot for Classification Results
def plot_classification_results(X, true_labels, predictions):
    plt.figure(figsize=(8, 6))
    plt.scatter(X[:, 0], X[:, 1], c=predictions, cmap='coolwarm', label='Predicted Classes', alpha=0.6)
    plt.scatter(X[:, 0], X[:, 1], marker='x', c=true_labels, cmap='coolwarm', label='True Classes', alpha=0.8)
    plt.xlabel('Feature 1')
    plt.ylabel('Feature 2')
    plt.title('Classification Results Scatter Plot')
    plt.legend(['Predicted', 'True'])
    plt.colorbar(label='Class Label')
    plt.show()

k = 3
# Get indices for phonemes 1 and 2
inds_1 = np.where(phoneme_id==1)[0]
inds_2 = np.where(phoneme_id==2)[0]
inds_1_and_2 = np.concatenate((inds_1, inds_2))

# Get feature data for both phonemes
X = X_full[inds_1_and_2]
N, D = X.shape

# Load GMM parameters for phoneme 1
p_id = 1
npy_filename = f'data/GMM_params_phoneme_{p_id:02}_k_{k:02}.npy'
if os.path.isfile(npy_filename):
    GMM_parameters_phoneme_1 = np.load(npy_filename, allow_pickle=True).tolist()
else:
    raise FileNotFoundError(f'File {npy_filename} does not exist.')

mu1, s1, p1 = GMM_parameters_phoneme_1['mu'], GMM_parameters_phoneme_1['s'], GMM_parameters_phoneme_1['p']

# Load GMM parameters for phoneme 2  
p_id = 2
npy_filename = f'data/GMM_params_phoneme_{p_id:02}_k_{k:02}.npy'
if os.path.isfile(npy_filename):
    GMM_parameters_phoneme_2 = np.load(npy_filename, allow_pickle=True).tolist()
else:
    raise FileNotFoundError(f'File {npy_filename} does not exist.')

mu2, s2, p2 = GMM_parameters_phoneme_2['mu'], GMM_parameters_phoneme_2['s'], GMM_parameters_phoneme_2['p']

# Calculate likelihoods for both models
Z_phoneme_1 = get_predictions(mu1, s1, p1, X)
likelihood_1 = np.sum(Z_phoneme_1, axis=1)

Z_phoneme_2 = get_predictions(mu2, s2, p2, X)
likelihood_2 = np.sum(Z_phoneme_2, axis=1)

# Make predictions using Maximum Likelihood
predictions = np.where(likelihood_2 > likelihood_1, 2, 1)

# Create true labels
true_labels = np.ones(N)
true_labels[len(inds_1):] = 2

# Calculate and display accuracy
accuracy = np.mean(predictions == true_labels)
print(f"Classification accuracy: {accuracy:.2%}")

# Visualize results
plot_classification_results(X, true_labels, predictions)

# Display confusion matrix
cm = confusion_matrix(true_labels, predictions)
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=[1, 2])
disp.plot(cmap='Blues', values_format='d')
plt.title('Confusion Matrix')
plt.show()

### <font color="red"> Q6. Repeat for $K=6$ and compare the results in terms of accuracy. [2 mark] </font>


In [ ]:
k = 6
inds_1 = np.where(phoneme_id==1)[0]
inds_2 = np.where(phoneme_id==2)[0]
inds_1_and_2 = np.concatenate((inds_1, inds_2))

X = X_full[inds_1_and_2,:]
N, D = X.shape

# Load GMM parameters for phoneme 1
p_id = 1
npy_filename = f'data/GMM_params_phoneme_{p_id:02}_k_{k:02}.npy'
if os.path.isfile(npy_filename):
    GMM_parameters_phoneme_1 = np.load(npy_filename, allow_pickle=True).tolist()
else:
    raise FileNotFoundError(f'File {npy_filename} does not exist.')

mu1, s1, p1 = GMM_parameters_phoneme_1['mu'], GMM_parameters_phoneme_1['s'], GMM_parameters_phoneme_1['p']

# Load GMM parameters for phoneme 2  
p_id = 2
npy_filename = f'data/GMM_params_phoneme_{p_id:02}_k_{k:02}.npy'
if os.path.isfile(npy_filename):
    GMM_parameters_phoneme_2 = np.load(npy_filename, allow_pickle=True).tolist()
else:
    raise FileNotFoundError(f'File {npy_filename} does not exist.')

mu2, s2, p2 = GMM_parameters_phoneme_2['mu'], GMM_parameters_phoneme_2['s'], GMM_parameters_phoneme_2['p']

# Calculate likelihoods for both models
Z_phoneme_1 = get_predictions(mu1, s1, p1, X)
likelihood_1 = np.sum(Z_phoneme_1, axis=1)

Z_phoneme_2 = get_predictions(mu2, s2, p2, X)
likelihood_2 = np.sum(Z_phoneme_2, axis=1)

# Make predictions using Maximum Likelihood
predictions = np.where(likelihood_2 > likelihood_1, 2, 1)

# Create true labels
true_labels = np.ones(N)
true_labels[len(inds_1):] = 2

# Calculate and display accuracy
accuracy = np.mean(predictions == true_labels)
print(f"Classification accuracy: {accuracy:.2%}")

# Visualize results
plot_classification_results(X, true_labels, predictions)

# Display confusion matrix
cm = confusion_matrix(true_labels, predictions)
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=[1, 2])
disp.plot(cmap='Blues', values_format='d')
plt.title('Confusion Matrix')
plt.show()

### <font color="red"> Q7. Display a "classification matrix" assigning labels to a grid of all combinations of the $F1$ and $F2$ features for the $K=3$ classifiers from above. Next, repeat this step for $K=6$ and compare the two. [3 marks] </font>

In particular,
create a custom uniform grid of points that spans all possible intermediate combinations of $F1$ and $F2$ features, using the features' minimum and maximum values as the limits. (Hint: check documentation for [np.meshgrid](https://numpy.org/doc/stable/reference/generated/numpy.meshgrid.html) and [np.linspace](https://numpy.org/doc/stable/reference/generated/numpy.linspace.html) ). Classify each point in the grid using one of your classifiers. That is, create a classification matrix, $\mathbf{M}$, whose elements are either 0 or 1. $M(i,j)$ is 0 if the point at location $(i,j)$ on the custom grid is classified as belonging to phoneme 1, and is 1 otherwise.

### Hint: building the custom grid:
Assuming our data were the orange points below, the custom grid would look something like the following (evenly spaced points spanning the minimum to maximum values of both features):

![](https://i.imgur.com/khfYx42.png)



### K=6

In [ ]:
k = 6  # Changed from 3 to 6 to test with more components

X = X_full[inds_1_and_2,:]

N = X.shape[0]
D = X.shape[1]

min_f1 = int(np.min(X[:,0]))
max_f1 = int(np.max(X[:,0]))
min_f2 = int(np.min(X[:,1]))
max_f2 = int(np.max(X[:,1]))
N_f1 = max_f1 - min_f1
N_f2 = max_f2 - min_f2
print('f1 range: {}-{} | {} points'.format(min_f1, max_f1, N_f1))
print('f2 range: {}-{} | {} points'.format(min_f2, max_f2, N_f2))

#########################################
# Create a finer grid with more points for smoother visualization
n_grid_points = 100
f1_grid, f2_grid = np.meshgrid(np.linspace(min_f1, max_f1, n_grid_points), 
                              np.linspace(min_f2, max_f2, n_grid_points))
grid_points = np.column_stack((f1_grid.ravel(), f2_grid.ravel()))

def get_predictions(mu, s, p, X):
    N = X.shape[0]
    K = len(p)
    D = X.shape[1]
    Z = np.zeros((N, K))
    
    for k in range(K):
        diff = X - mu[k]
        if s[k].ndim == 2:  # If s[k] is a 2D array (covariance matrix)
            inv_cov = np.linalg.inv(s[k])
            mahalanobis = np.sum(np.dot(diff, inv_cov) * diff, axis=1)
        else:  # If s[k] is a 1D array (diagonal of covariance matrix)
            mahalanobis = np.sum((diff / s[k])**2, axis=1)
        
        Z[:, k] = p[k] * np.exp(-0.5 * mahalanobis) / np.sqrt((2*np.pi)**D * np.linalg.det(s[k]))
    
    return Z

# Get GMM parameters
mu1 = GMM_parameters_phoneme_1['mu']
s1 = GMM_parameters_phoneme_1['s'] 
p1 = GMM_parameters_phoneme_1['p']

mu2 = GMM_parameters_phoneme_2['mu']
s2 = GMM_parameters_phoneme_2['s']
p2 = GMM_parameters_phoneme_2['p']

# Perform predictions
Z_phoneme_1 = get_predictions(mu1, s1, p1, grid_points)
Z_phoneme_2 = get_predictions(mu2, s2, p2, grid_points)

# Calculate likelihoods
likelihood_1 = np.sum(Z_phoneme_1, axis=1)
likelihood_2 = np.sum(Z_phoneme_2, axis=1)

# Classify points based on higher likelihood (0 for phoneme 1, 1 for phoneme 2)
M = (likelihood_2 > likelihood_1).reshape(n_grid_points, n_grid_points).astype(float)

# Visualize predictions on custom grid
fig, ax = plt.subplots(figsize=(10, 8))

# Plot the classification matrix
im = plt.imshow(M, aspect='auto', extent=[min_f1, max_f1, min_f2, max_f2], origin='lower')

# Set labels and title
ax.set_xlabel('F1')
ax.set_ylabel('F2') 
plt.title(f'Classification Matrix (K={k})')

# Add colorbar
plt.colorbar(im, label='Predicted Class')

# Plot the original data points
N_samples = int(X.shape[0]/2)
plt.scatter(X[:N_samples, 0], X[:N_samples, 1], marker='.', color='red', 
           label='Phoneme 1', alpha=0.5, s=10)
plt.scatter(X[N_samples:, 0], X[N_samples:, 1], marker='.', color='green', 
           label='Phoneme 2', alpha=0.5, s=10)

plt.legend()
plt.show()

### K=3


In [ ]:
k = 3

X = X_full[inds_1_and_2,:]

N = X.shape[0]
D = X.shape[1]

min_f1 = int(np.min(X[:,0]))
max_f1 = int(np.max(X[:,0]))
min_f2 = int(np.min(X[:,1]))
max_f2 = int(np.max(X[:,1]))
N_f1 = max_f1 - min_f1
N_f2 = max_f2 - min_f2
print('f1 range: {}-{} | {} points'.format(min_f1, max_f1, N_f1))
print('f2 range: {}-{} | {} points'.format(min_f2, max_f2, N_f2))

#########################################
# Create a finer grid with more points for smoother visualization
n_grid_points = 100
f1_grid, f2_grid = np.meshgrid(np.linspace(min_f1, max_f1, n_grid_points), 
                              np.linspace(min_f2, max_f2, n_grid_points))
grid_points = np.column_stack((f1_grid.ravel(), f2_grid.ravel()))

def get_predictions(mu, s, p, X):
    N = X.shape[0]
    K = len(p)
    D = X.shape[1]
    Z = np.zeros((N, K))
    
    for k in range(K):
        diff = X - mu[k]
        if s[k].ndim == 2:  # If s[k] is a 2D array (covariance matrix)
            inv_cov = np.linalg.inv(s[k])
            mahalanobis = np.sum(np.dot(diff, inv_cov) * diff, axis=1)
        else:  # If s[k] is a 1D array (diagonal of covariance matrix)
            mahalanobis = np.sum((diff / s[k])**2, axis=1)
        
        Z[:, k] = p[k] * np.exp(-0.5 * mahalanobis) / np.sqrt((2*np.pi)**D * np.linalg.det(s[k]))
    
    return Z

# Get GMM parameters
mu1 = GMM_parameters_phoneme_1['mu']
s1 = GMM_parameters_phoneme_1['s'] 
p1 = GMM_parameters_phoneme_1['p']

mu2 = GMM_parameters_phoneme_2['mu']
s2 = GMM_parameters_phoneme_2['s']
p2 = GMM_parameters_phoneme_2['p']

# Perform predictions
Z_phoneme_1 = get_predictions(mu1, s1, p1, grid_points)
Z_phoneme_2 = get_predictions(mu2, s2, p2, grid_points)

# Calculate likelihoods
likelihood_1 = np.sum(Z_phoneme_1, axis=1)
likelihood_2 = np.sum(Z_phoneme_2, axis=1)

# Classify points based on higher likelihood (0 for phoneme 1, 1 for phoneme 2)
M = (likelihood_2 > likelihood_1).reshape(n_grid_points, n_grid_points).astype(float)

# Visualize predictions on custom grid
fig, ax = plt.subplots(figsize=(10, 8))

# Plot the classification matrix
im = plt.imshow(M, aspect='auto', extent=[min_f1, max_f1, min_f2, max_f2], origin='lower')

# Set labels and title
ax.set_xlabel('F1')
ax.set_ylabel('F2') 
plt.title(f'Classification Matrix (K={k})')

# Add colorbar
plt.colorbar(im, label='Predicted Class')

# Plot the original data points
N_samples = int(X.shape[0]/2)
plt.scatter(X[:N_samples, 0], X[:N_samples, 1], marker='.', color='red', 
           label='Phoneme 1', alpha=0.5, s=10)
plt.scatter(X[N_samples:, 0], X[N_samples:, 1], marker='.', color='green', 
           label='Phoneme 2', alpha=0.5, s=10)

plt.legend()
plt.show()

Create a new dataset that will contain 3 columns, as follows:
\begin{equation}
    X = [F1, F2, F1+F2]
\end{equation}

### <font color="red"> Q8. Try to fit a MoG model to the new data. What is the problem that you observe? Explain why it occurs [2 marks]

In [ ]:
# data = np.ndarray.tolist(data)
# Array that contains the phoneme ID (1-10) of each sample
phoneme_id = data['phoneme_id']

# frequencies f1 and f2
f1 = data['f1']
f2 = data['f2']
print(phoneme_id)

In [ ]:

p_id = 2 # id of the phoneme that will be used (e.g. 1, or 2)

k = 3 # number of GMM components
X = np.zeros((len(f1), 3))
#########################################
X[:, 0] = f1  # First column is f1
X[:, 1] = f2  # Second column is f2
X[:, 2] = f1 + f2  # Third column is f1+f2
print("Shape of X before filtering:", X.shape)
phoneme_mask = (phoneme_id == p_id)
print("Shape of X after filtering:", X.shape)
print("First few rows of X:")
print(X[:5])

#########################################
fig = plt.figure()
ax1 = plt.axes(projection='3d')
title_string = 'Phoneme {}'.format(p_id)
plot_data_3D(X=X, title_string=title_string, ax=ax1)

N, D = X.shape
p = np.ones((k))/k
random_indices = np.floor(N*np.random.rand((k)))
random_indices = random_indices.astype(int)
mu = X[random_indices,:] # shape kxD
s = np.zeros((k,D,D)) # shape kxDxD
n_iter = 150

# initialize covariances
for i in range(k):
    cov_matrix = np.cov(X.transpose())
    s[i,:,:] = cov_matrix/k

Z = np.zeros((N,k)) # shape Nxk

###############################
# run Expectation Maximization algorithm for n_iter iterations
for t in range(n_iter):
    #print('****************************************')
    print('Iteration {:03}/{:03}'.format(t+1, n_iter))

    # Do the E-step
    Z = get_predictions(mu, s, p, X)
    Z = normalize(Z, axis=1, norm='l1')
    # Do the M-step:
    for i in range(k):
        mu[i,:] = np.matmul(X.transpose(), Z[:,i])/np.sum(Z[:,i])

        ###################################################
        # We will fit Gaussians with full covariance matrices:
        mu_i = mu[i,:]
        mu_i = np.expand_dims(mu_i, axis=1)
        mu_i_repeated = np.repeat(mu_i, N, axis=1)

        t_1 = X.transpose() - mu_i_repeated
        t_2 = np.repeat(np.expand_dims(Z[:,i], axis=1), D, axis=1) * t_1.transpose()
        s[i,:,:] = np.matmul(t_1, t_2)/np.sum(Z[:,i])
        p[i] = np.mean(Z[:,i])

plot_gaussians(ax1, 2*s, mu, X, 'Phoneme {}'.format(p_id))
plt.show()

### <font color="red"> Q9. Suggest ways of overcoming the singularity problem and implement one of them. Show any training outputs in the report and discuss. [3 marks] </font>

In [ ]:
p_id = 1
k = 3
X = np.zeros((len(f1), 3))

X[:, 0] = f1
X[:, 1] = f2
X[:, 2] = f1 + f2
X = X[phoneme_id == p_id]

# Standardize the data to help with numerical stability
X = (X - np.mean(X, axis=0)) / np.std(X, axis=0)

fig = plt.figure()
ax1 = plt.axes(projection='3d')
title_string = 'Phoneme {}'.format(p_id)
plot_data_3D(X=X, title_string=title_string, ax=ax1)

N, D = X.shape
p = np.ones((k)) / k
random_indices = np.floor(N * np.random.rand((k))).astype(int)
mu = X[random_indices, :]
s = np.zeros((k, D, D))
n_iter = 150

# Initialize covariances with regularization
reg_param = 1e-3  # Increased regularization parameter
for i in range(k):
    cov_matrix = np.cov(X.transpose())
    s[i, :, :] = (cov_matrix / k) + (reg_param * np.eye(D))

Z = np.zeros((N, k))

# Add convergence check
prev_likelihood = -np.inf
convergence_threshold = 1e-6

for t in range(n_iter):
    print('Iteration {:03}/{:03}'.format(t + 1, n_iter))

    # E-step with numerical stability checks
    Z = get_predictions(mu, s, p, X)
    Z = np.clip(Z, 1e-10, None)  # Prevent zero probabilities
    Z = normalize(Z, axis=1, norm='l1')

    # M-step with robust updates
    for i in range(k):
        # Update means
        weight_sum = np.sum(Z[:, i])
        if weight_sum > 1e-10:  # Only update if enough data points are assigned
            mu[i, :] = np.matmul(X.transpose(), Z[:, i]) / weight_sum

            # Update covariances with robust computation
            mu_i = np.expand_dims(mu[i, :], axis=1)
            mu_i_repeated = np.repeat(mu_i, N, axis=1)
            
            t_1 = X.transpose() - mu_i_repeated
            t_2 = np.repeat(np.expand_dims(Z[:, i], axis=1), D, axis=1) * t_1.transpose()
            s[i, :, :] = np.matmul(t_1, t_2) / weight_sum + (reg_param * np.eye(D))
            
            # Ensure positive definiteness
            eigvals = np.linalg.eigvals(s[i, :, :])
            if np.any(eigvals < reg_param):
                s[i, :, :] += (abs(np.min(eigvals)) + reg_param) * np.eye(D)
            
            p[i] = weight_sum / N

    # Check for convergence
    current_likelihood = np.sum(np.log(np.sum(Z, axis=1)))
    if abs(current_likelihood - prev_likelihood) < convergence_threshold:
        print(f"Converged after {t+1} iterations")
        break
    prev_likelihood = current_likelihood

plot_gaussians(ax1, 2 * s, mu, X, 'Phoneme {}'.format(p_id))
plt.show()